# Can hitters change their behavior in different game situations? #

Name: Michael Dresser

In recent years, there has a proliferation of defensive shifts in baseball. Teams identify a hitter's tendencies, and then change their positioning on the field to maximize their chances of getting the hitter out. We've seen this in the most extreme where the right side of the infield is overloaded - the 3rd baseman plays at the shortstop position, the shortstop is positioned behind 2nd, and the 2nd baseman is in short right field.

The obvious question for a fan seeing a defensive shift is, why don't hitters just hit it to the 3rd base side and take the single? Instead, hitters often hit the ball right into the shift. As long as the shift is working, teams will continue to use it to increase their chances of getting hitters out.

There is another question that arises about hitters - can hitters change their natural tendencies to match the game situation? For example, can hitters who don't hit a lot of fly balls hit a fly ball with a runner on 3rd and no outs to get the sacrifice? Can natural fly ball hitters hit a ground ball to the right side with a runner on 2nd and no outs? There are scenarios when a hitters behavior could have a benefit for the team's ability to score a run.

There is current trend of applying a defensive shift, primarily for left-handed pull hitters. The result is a defensive alignment where the 3rd baseman plays at the shortstop position, the shortstop is behind 2nd base in short center field, and the 2nd baseman is shifted over toward 1st base in short right field. The obvious question is, why don't hitters just adjust their tendencies and hit the ball somewhere else?

## Situational performance scenarios ##
For any game situation, are some players better than other at adjusting their behavior to match the situation? Alternatively, we could ask if there is a change in behavior, whether intentional or not.

* Rockies fans watched for years as Troy Tulowitski came to bat with runners on base and hit a long fly ball out. It's hard not to wonder if he's trying to hit a home run. Whether intentional or unintentional, can we determine if his behavior changes with runners on base, especially in late innings with the game on the line? Is he trying to be the hero?

* With a runner on third and less than two outs, it's nice to see that run score. Yet, batters aren't always successful in hitting a fly ball in that situation. Can we determine if some batters are more likely to hit a fly ball in that situation than they would normally?

* With a runner on 2nd and no outs, getting the runner to 3rd means that the runner scores on a sacrifice fly. In a close game, moving this runner could be key to scoring a winning or tying run. Can we determine if some batters can hit a right-side ground ball in this situation even if they don't normally hit ground balls to the right side?



### Challenges ###
* Using standard statistical methods requires knowing the underlying distribution.
* Distribution might not be normal
* Different sample sizes in different situations skew the results.

# Bootstrapping #
We're going to use a technique called bootstrapping to answer questions about players' behavior in different situations. We could also use this approach to determine run production at Coors Field is an outlier compared to other ballparks. 

* Bootstrapping is a technique where we simulate the outcome of a process by sampling randomly from a data set. 

* We use these samples to generate a distribution. 

* We don't need to know the parameters of the distribution, such as mean, std, and median ahead of time, we generate them from our sampling.

* Works with small datasets.

* Why is it called Bootstrapping? There's an expression, "Pull yourself up by your bootstraps." In statistics, it's accomplishing what you need with what you've got without sacrificing statistical rigor.

## A non-baseball example ##
Let's say you're sitting outside the engineering center one day and you see 10 people walk by, wearing different color jackets. You see,

* 5 black jackets
* 2 red jackets
* 1 blue jacket
* 2 green jacket

From these observations, you wonder what the probability is of seeing a student at CU wearing a red jacket. Your first thought is that it's 20\%. But, your sample size is so small and there are 30K students at CU. What are the mean, std of your observations? 

The information that you have is the empirical distribution of the data, not the underlying distribution. 

We can use bootstrapping and approximate the underlying distribution from these 10 observations by simulating the process of observing 10 other people, many times. With enough samples, the empirical distribution matches the underlying distribution.

We randomly draw from the possible values uniformly. In this example, $n = 10$, we can create an array of the 4 distinct values observed in the $n$ observations:
$$[B, B, B, B, B, R, R, BL, G, G]$$

__Bootstrap Resample:__ A set of $n$ draws from the original sample with replacement.

__What does sample with replacement mean?__

__Draw:__ A draw is a process of selecting one element randomly from our original sample.

### Bootstrap Resample ###
To generate a bootstrap resample in this example, we generate a random number, 1-10, that represents a position in our array of observations. For example, if we randomly generate the numbers 3 and 6, that means that we observed a B and R jacket because they are in the 3 and 6 positions in our array. We need $n$ draws for a complete resample. 

Assume we generate the following draws:

$$[4, 6, 7, 3, 6, 5, 10, 1, 2, 9]$$

it is equivalent to observing:

$$[B, R, R, B, R, B, G, B, B, G]$$

Three of the observations are R, which is equivalent to observing 3/10 red jackets. To generate a distribution of observations, we repeat the resampling process many times, let's say 1,000. We have 1,000 iterations of $n$ draws, and in each of the $n$ draws, there is a number of times that B is observed.

We can implement this process in Python pretty easily. In one run of 1000 resamples, I produced the following values:

* 0-118
* 1-275
* 2-309
* 3-192
* 4-74
* 5-19
* 6-7
* 7-1
* 8-0
* 9-0
* 10-0

The descriptive statistics for this distribution include:

* count=1000.000000
* mean=2.066000
* std=1.308953
* min=0.000000
* 25\%=1.000000
* 50\%=2.000000
* 75\%=3.000000
* max=7.000000


In [39]:
from random import randint
import pandas as pd 
#array of jacket values
jackets = [0,0,1,2,3]
trials = [0] * 1000
for i in range(0,1000): #1000 trials
    countForTrial = 0
    for j in range(0,10): #10 jackets observed, get their color
        x = randint(0,9)
        if x == 5 or x == 6:
            countForTrial += 1

    trials[i] = countForTrial

trials.sort()
df = pd.DataFrame(trials)
print(df.describe())

#What do the values mean here?

                 0
count  1000.000000
mean      1.940000
std       1.235284
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       6.000000


## A Baseball Example ##
Tulo...Tulo...Tulo. Did Troy Tulowitski hit more flyballs with a runner on third and less than two outs than he did in other base-out situations, with less than two outs? 

We can't know a player's intent. But, one would hope that getting that run to score from third would be a priority for the batter, or at least that's what old-school baseball fans would say.

From the data, can we determine if there was a difference in the distribution of flyballs for Tulo in this situation?

### What data do we need? ###
* The number of times that Tulo batted with a runner on third and less than two outs, and how many times that resulted in a fly ball.
* The number of times that Tulo batted with no runner on third and less than two outs, and how many times that resulted in a fly ball.

#### Data results ####

* 143 times tulo batted with a runner on third and less than 2 outs
  - 16 times tulo hit a fly-ball out to the outfield

* 1087 times tulo batted with no runner on third and less than 2 outs
  - 132 times tulo hit a fly-ball out to the outfield with no runner on third

__Null Hypothesis: There is no change in Tulo's behavior with a runner on third when there is less than 2 outs.__

* The pool of possible values is 143+1087=1230 at-bats. 
  - 16 of 143 is flyball and 132 of 1087 is flyball.

* A draw is an at-bat. The result of the draw is he either hits a flyball or he doesn't.  

* A sample is 143 at-bats for Tulo to simulate the times he came to bat with runner on third and less than 2 outs.

#### Setting up the simulation ####
* In all of his at-bats, how many times did Tulo hit a flyball out? 148

* Need a list of 1230 at-bats, where 0-147 = 1 for flyball out and 148-1229 = 0 for not flyball out

### Simulation results ###
In one run of the simulation, 16 of 143 shows up in 96 out of 1000 trials, so p-value of observing 16 flyballs in 143 at bats is .096. Other runs of the simulation produced similar results. We can't reject the null hypothesis. There is no change in his flyball hitting behavior with a runner on third. 


In [40]:
from random import randint
import pandas as pd 
import matplotlib.pylab as plt

#tulo and the flyball out
hits = [0] * 1230
for i in range(0,148):
    hits[i] = 1
trials = [0] * 1000
for i in range(0,1000):
    countForTrial = 0
    for j in range(0,143):
        x = randint(0,1229)
        countForTrial+=hits[x]
    trials[i] = countForTrial
trials.sort()
tr = 0
for t in trials:
    if t == 16:
        tr+=1
print(tr/1000)
df = pd.DataFrame(trials)
print(df.describe())
# Initialize figure and axis
fig, ax = plt.subplots(figsize=(8,4))

# Plot histogram 
df.hist(ax=ax);
# Add a title
ax.set_title("Tulo flyball outs", fontsize=20)

# Add axis labels 
ax.set_xlabel("Number of flyballs in 143 at-bats", fontsize=16)
ax.set_ylabel("Frequency", fontsize=16)

0.119
                 0
count  1000.000000
mean     17.233000
std       3.874238
min       5.000000
25%      15.000000
50%      17.000000
75%      20.000000
max      30.000000


Text(0,0.5,'Frequency')

## Questions ##
1. (2 pts) Explain how you could use bootstrapping to determine how run scoring at Coors Field compares to the rest of the National League? Specifically, how does the mean number of runs scored at Coors in the 2018 season compare to the mean number of runs scored at other stadiums?
   
   1a. What is a draw for this problem?
   
   1b. What is a sample?
   
   1c. How do you set up your simulation?
   
   1d. How would you determine if run scoring at Coors is a statistically significant outlier?
   
   1e. What is the Null hypothesis?
   
2. (2 pts) Choose your favorite player who had at least 500 plate appearances in 2018. How would you determine if he had a statistically significant percentage of strike outs with runners on second and third and no outs than he did with bases empty and no outs?     
   
   2a. What is a draw for this problem?
   
   2b. What is a sample?
   
   2c. How do you set up your simulation?
   
   2d. What is the Null hypothesis?

3. (2 pts) Choose either problem 1 or 2 and implement it and explain the result that you get.

1a. Runs scored in one game at a park  
1b. A sample is a number of draws for one park equal to the number of games played at Coors in a season.  
1c. Simulation: pull the scores from every game for every park. Resample scores for Coors many times and calculate the mean each time. Do the same but for all non-Coors NL parks. Compare mean distrubtions.  
1d. The mean number of runs scored at Coors in 2018 is no different from the mean number of runs scored at other NL parks in 2018.

2a. A draw is an at-bat with runners on second and third and no outs OR an at-bat with bases empty and no outs.  
2b. A sample is the results of a number of draws equal to the times the player came up with runners on second and third and no outs OR equal to the number of times the player came up with bases empty and no outs.  
2c. Simulation: find all at-bats for both situations (separately). Calculate the number of ABs in each situation that resulted in a SO and the number that did not. This forms the data to draw from in each situation. The simulation process is then simple for each situation. Then compare the means.    
2d. There is no difference between the strike out percentage in the two situations.  

In [41]:
from pybaseball import playerid_lookup
playerid_lookup("chapman", "matt")

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,chapman,matt,-1,NaN,NaN,-1,NaN,NaN
1,chapman,matt,656305,chapm001,chapmma01,16505,2017.0,2019.0


In [42]:
playerid_lookup("davis", "khris")

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,davis,khris,501981,davik003,daviskh01,9112,2013.0,2019.0


In [43]:
playerid_lookup("semien")

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,semien,marcus,543760,semim001,semiema01,12533,2013.0,2019.0


In [45]:
import pandas as pd
from pybaseball import batting_stats
statcast = pd.read_csv("pitches_2016_2018.csv")
statcast = statcast.loc[(statcast["game_date"] < "2019-01-01") & (statcast["game_date"] > "2018-01-01")]
b = batting_stats(2018)

In [46]:
b.loc[(b["Name"] == "Marcus Semien")]["AB"]

319    632.0
Name: AB, dtype: float64

In [47]:
df_p = statcast.loc[(statcast["batter"] == 543760)]

In [48]:
w_runners = df_p.loc[(df_p["on_3b"].notna()) &
                     (df_p["on_2b"].notna()) &
                     (df_p["on_1b"].isna()) &
                     (df_p["outs_when_up"] == 0)
                    ].dropna(subset=["events"])

w_runners_so = len(w_runners.loc[w_runners["events"] == "strikeout"])
w_runners_notso = len(w_runners.loc[w_runners["events"] != "strikeout"])

In [49]:
wo_runners = df_p.loc[(df_p["on_3b"].isna()) &
                      (df_p["on_2b"].isna()) &
                      (df_p["on_1b"].isna()) &
                      (df_p["outs_when_up"] == 0)
                     ].dropna(subset=["events"])

wo_runners_so = len(wo_runners.loc[wo_runners["events"] == "strikeout"])
wo_runners_notso = len(wo_runners.loc[wo_runners["events"] != "strikeout"])

In [50]:
# 1 if draw returned SO
def draw_w_runners():
    x = randint(1, w_runners_so + w_runners_notso)
    if x <= w_runners_so:
        return 1
    else:
        return 0
    
# 1 if draw returned SO
def draw_wo_runners():
    x = randint(1, wo_runners_so + wo_runners_notso)
    if x <= wo_runners_so:
        return 1
    else:
        return 0

In [53]:
def get_sample_w_runners():
    return [draw_w_runners() for _ in range(w_runners_so + w_runners_notso)]

def get_sample_wo_runners():
    return [draw_wo_runners() for _ in range(wo_runners_so + wo_runners_notso)]

In [87]:
import numpy as np
def bootstrap_mean_w(trials=10000):
    t = np.zeros(trials)
    for i in range(trials):
        t[i] = np.mean(get_sample_w_runners())
    return t

def bootstrap_mean_wo(trials=10000):
    t = np.zeros(trials)
    for i in range(trials):
        t[i] = np.mean(get_sample_wo_runners())
    return t

In [91]:
trials = 10000
w_runners_sorate_means = bootstrap_mean_w(trials)
wo_runners_sorate_means = bootstrap_mean_wo(trials)

In [82]:
from scipy.stats import ttest_ind

In [93]:
ttest_ind(w_runners_sorate_means, wo_runners_sorate_means)

Ttest_indResult(statistic=-16.0119040215061, pvalue=2.3987601300868855e-57)

Results: the p-value and statistic suggests that there is a difference, but I'm not sure that I trust it, given how small the sample is for the runners on situation.